---

## LSTM Model v2

---

Multivariate usage of an LSTM.

In [ ]:
from datetime import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.layers import Dense,RepeatVector, LSTM, Dropout
from keras.layers import Flatten, Conv1D, MaxPooling1D
from keras.layers import Bidirectional, Dropout
from keras.models import Sequential
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

In [ ]:
class ModelLSTMv2:
  """
  """
  def __init__(self, window_size=30, num_epochs=300, debug=False):
    self.WINDOW_SIZE = window_size
    self.NUM_EPOCHS = num_epochs
    self.debug = debug

    self.MODEL_NAME = "LSTMv2"

    if self.debug:
      print(f'### Building Model{self.MODEL_NAME}::')


  def train(self, X_train=None, y_train=None, num_features=None, dataset=None):
    """
    Build and train the model.
    """
    early_stop = EarlyStopping(monitor = "loss", mode = "min", patience = 25)
    model = Sequential()
    model.add(Conv1D(filters=256, kernel_size=2, activation='relu', input_shape=(self.WINDOW_SIZE, num_features)))
    model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(RepeatVector(self.WINDOW_SIZE))
    model.add(LSTM(units=100, return_sequences=True, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(LSTM(units=100, return_sequences=True, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(LSTM(units=100, return_sequences=True, activation='tanh'))
    model.add(LSTM(units=100, return_sequences=True, activation='tanh'))
    model.add(Bidirectional(LSTM(128, activation='tanh')))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')

    if (dataset is not None):

      self.model_hist = model.fit(dataset, epochs=self.NUM_EPOCHS, callbacks = [early_stop], verbose=(1 if self.debug else 0))
    else:
      self.model_hist = model.fit(X_train, y_train, epochs=self.NUM_EPOCHS, verbose=1, callbacks = [early_stop] )
    self.model = model
    return self.model_hist

  def get_model_name(self):
    return f"{dt.today().strftime('%Y%m%d-%H%M')}-{self.MODEL_NAME}.hdf5"

  def save_model(self, path):
    """
    Save the current model under the given drive path.
    Timestamp the model name.
    """
    fname = f'{path}{self.get_model_name()}'
    print(f'Saving model to: {fname}')
    return self.model.save(fname)

  def predict(self, X_in):
    return self.model.predict(X_in)